In [1]:
import numpy as np


import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
import torchvision
import pathlib

In [2]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(torch.cuda.is_available())

True


In [3]:
#transforms
transformer=transforms.Compose([
    transforms.Lambda(lambda img: img.convert('RGB')),
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(), #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5],  #0-1 to [-1,1], formula x-mean/std
                         [0.5,0.5,0.5])    
])

In [7]:
#Path to directories
train_path="C:/Users/HP/Desktop/Extras/PythonML/CatClassifierSmall/dataset/train"
test_path="C:/Users/HP/Desktop/Extras/PythonML/CatClassifierSmall/dataset/test"
train_loader=DataLoader(torchvision.datasets.ImageFolder(train_path,transform=transformer),batch_size=128, shuffle = True)
test_loader=DataLoader(torchvision.datasets.ImageFolder(test_path,transform=transformer),batch_size=128, shuffle = True)


In [8]:

 #categoires
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [9]:

#CNN network
class ConvNet(nn.Module):
    def __init__(self,num_classes=2):
        super(ConvNet,self).__init__()

        #OUTPUT SIZE AFTER CONVOLUTION FILETER
        # ((w-f+2P)/s)+1

        #input shape =(256,3,150,150)
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #shape=(256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #shape=(256,12,150,150)
        self.relu1=nn.ReLU()
        #shape=(256,12,150,150)

        self.pool=nn.MaxPool2d(kernel_size=2)
        #reduce the image size be factor 2
        #shape=(256,12,75,75)

        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #shape=(256,20,75,75)
        self.relu2=nn.ReLU()
        #shape=(256,22,75,75)

        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #shape=(256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #shape=(256,32,75,75)
        self.relu3=nn.ReLU()
        #shape=(256,32,75,75)

        self.fc=nn.Linear(in_features=32*75*75,out_features=num_classes)

    #feed forward function
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
        
        output=self.pool(output)
        output=self.conv2(output)
        output=self.relu2(output)
        
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)

        #above output will be in matrix form with shape(256,32,75,75)
        output=output.view(-1,32*75*75)

        output=self.fc(output)

        return output

In [10]:
model=ConvNet(num_classes=2).to(device)

In [11]:

#optimizer and loss function
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()


In [12]:
num_epochs=10

In [13]:
#calculation the size af training and testing images
train_count=len(glob.glob(train_path+'/**/*'))
test_count=len(glob.glob(test_path+'/**/*'))


In [15]:
test_count

1873

In [14]:
#model training and savin best model

best_accuracy=0
for epoch in range(num_epochs):
    #evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    for i,(images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=images.to(device)
            labels=labels.to(device)

        
        optimizer.zero_grad()
        outputs=model(images)

        

        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()

        train_loss+=loss.cpu()*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        print("Train loss: ",i,train_loss)
        i+=1
        train_accuracy+=int(torch.sum(prediction==labels.data))

    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    #evaluaiton on testinf datasert
    model.eval()
    test_accuracy=0.0
    for i,(images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=images.to(device)
            labels=labels.to(device)

        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))

    test_accuracy=test_accuracy/test_count

    print('Epoch: '+str(epoch)+' Train Loss: '+str(int(train_loss))+' Train Accuracy: '+ str(train_accuracy)+' Test accuracy: '+str(test_accuracy))

    #save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy 

C:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Train loss:  0 tensor(85.8664, grad_fn=<AddBackward0>)
Train loss:  1 tensor(3944.7561, grad_fn=<AddBackward0>)
Train loss:  2 tensor(6539.3496, grad_fn=<AddBackward0>)
Train loss:  3 tensor(9334.1719, grad_fn=<AddBackward0>)
Train loss:  4 tensor(11853.8789, grad_fn=<AddBackward0>)
Train loss:  5 tensor(15033.0908, grad_fn=<AddBackward0>)
Train loss:  6 tensor(16672.2012, grad_fn=<AddBackward0>)
Train loss:  7 tensor(17355.2988, grad_fn=<AddBackward0>)
Train loss:  8 tensor(18752.1133, grad_fn=<AddBackward0>)
Train loss:  9 tensor(21020.7227, grad_fn=<AddBackward0>)
Train loss:  10 tensor(22290.8711, grad_fn=<AddBackward0>)
Train loss:  11 tensor(22883.4746, grad_fn=<AddBackward0>)
Train loss:  12 tensor(23717.8223, grad_fn=<AddBackward0>)
Train loss:  13 tensor(24561.0156, grad_fn=<AddBackward0>)
Train loss:  14 tensor(24870.6328, grad_fn=<AddBackward0>)
Epoch: 0 Train Loss: 13 Train Accuracy: 0.643440391943386 Test accuracy: 0.3539775760811532
Train loss:  0 tensor(578.0392, grad_fn